<a href="https://colab.research.google.com/github/RaghavMaskara21/Machine-Learning/blob/main/5_method_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install pyspellchecker
!apt update
!apt install -qq enchant
!pip install pyenchant
!pip3 install https://github.com/andreasvc/readability/tarball/master
import pandas as pd
import numpy as np
import io
from __future__ import unicode_literals
import spacy,en_core_web_sm
from collections import Counter
from spellchecker import SpellChecker    
import enchant
import readability
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from enchant.checker import SpellChecker

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
stop_words = set(stopwords.words('english'))
pd.set_option('mode.chained_assignment', None)

In [5]:
def extractFeature(text,index):
  c = Counter(([token.pos_ for token in nlp(text)]))
  data['Nouns'][index] = c['NOUN'] if 'NOUN' in c else 0
  data['Verbs'][index] = c['VERB'] if 'VERB' in c else 0
  data['ProperNouns'][index] = c['PROPN'] if 'PROPN' in c else 0
  data['Adjectives'][index] = c['ADJ'] if 'ADJ' in c else 0
  data['Punctuation'][index] = c['PUNCT'] if 'PUNCT' in c else 0
  
  # get wrong words
  wordlist=text.split()
  chkr = SpellChecker("en_US",text)
  data['WrongWords'][index] = len(list(chkr))
  d = enchant.Dict("en_US")
  data['DifficultWords'][index]= [d.check(w) for w in wordlist].count(False)
  data['Length'][index] = len(text)
  
  data['StopWords'][index] = len(set(wordlist) & stop_words)
  data['OneLetterWords'][index]=len([word for word in wordlist if len(word)==1])
  data['TwoLetterWords'][index]=len([word for word in wordlist if len(word)==2])

  #get readability index
  results = readability.getmeasures(text, lang='en')
  data['FleschReading'][index] = results['readability grades']['FleschReadingEase']
  data['DaleChallIndex'][index] = results['readability grades']['DaleChallIndex']


In [6]:
data = pd.read_csv("spam.csv", encoding = "latin-1")
data = data.dropna(how = "any", axis = 1)
nlp = en_core_web_sm.load()
index = 0
data['Nouns'] = 0
data['ProperNouns'] = 0
data['Verbs'] = 0
data['Adjectives'] = 0
data['Punctuation'] = 0
data['WrongWords'] = 0
data['StopWords'] = 0
data['Length'] = 0
data['OneLetterWords'] = 0
data['TwoLetterWords'] = 0
data['DifficultWords'] = 0
data['FleschReading'] = 0
data['DaleChallIndex'] = 0
for text in data['v2']:  
  extractFeature(text,index)
  #print(index)
  index+=1
data
data.to_csv('spam_features.csv')
